In [1]:
# Imports  
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# Get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2020-11-22 12:16:02--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip      [       <=>          ]  24.88M  18.8MB/s    in 1.3s    

2020-11-22 12:16:04 (18.8 MB/s) - ‘book-crossings.zip’ saved [26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# Importing csv data into pd dataframes
def csv_to_df():
  df_books = pd.read_csv(
      books_filename,
      encoding = "ISO-8859-1",
      sep=";",
      header=0,
      names=['isbn', 'title', 'author'],
      usecols=['isbn', 'title', 'author'],
      dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

  df_ratings = pd.read_csv(
      ratings_filename,
      encoding = "ISO-8859-1",
      sep=";",
      header=0,
      names=['user', 'isbn', 'rating'],
      usecols=['user', 'isbn', 'rating'],
      dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})
  return df_books, df_ratings

In [4]:
# Cleaning the data
def clean_data(df_books, df_ratings):

  # Removing null data
  df_books.dropna(inplace = True)
  
  # Removing users with less than 200 ratings
  user_ratings = df_ratings['user'].value_counts()
  user_ratings.sort_values(ascending = False)
  df_ratings_removed = df_ratings[ ~df_ratings['user'].isin(user_ratings[user_ratings < 200].index)]
  
  # Removing books with less than 100 ratings
  book_ratings = df_ratings['isbn'].value_counts()
  book_ratings.sort_values(ascending = False).head
  df_ratings_removed = df_ratings_removed[
  ~df_ratings_removed['isbn'].isin(book_ratings[book_ratings < 100].index)]
  return df_ratings_removed

In [5]:
# Preparing the data for the knn model
def prepare_data(df_ratings_removed):
  df = df_ratings_removed.pivot_table(index=['user'],columns=['isbn'],values='rating').fillna(0).T
  df.index = df.join(df_books.set_index('isbn'))['title']
  df = df.sort_index()
  return df

In [6]:
# Creating and training the knn model
def create_and_train_knn_model(df):
  knn_model = NearestNeighbors(metric='cosine')
  knn_model.fit(df.values)
  return knn_model

In [7]:
# Recommendation function
def get_recommends(df, knn_model, book_title):
  try:
    book = df.loc[book_title]
  except KeyError as book_error:
    print('The entered book', book_error, 'does not exist')
    return

  distance, indice = knn_model.kneighbors([book.values], n_neighbors=6)

  recommended_books = pd.DataFrame({
      'title'   : df.iloc[indice[0]].index.values,
      'distance': distance[0]
    }) \
    .sort_values(by='distance', ascending=False) \
    .head(5).values

  return [book_title, recommended_books]

In [8]:
# Function to test if the challenge is passed or not
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends(df, knn_model, "Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")



In [9]:
if __name__ == '__main__':
  df_books, df_ratings = csv_to_df()
  df_ratings_removed = clean_data(df_books, df_ratings)
  df = prepare_data(df_ratings_removed)
  knn_model = create_and_train_knn_model(df)
  book_title = input("Enter a book name : ")
  recommendations = get_recommends(df, knn_model, book_title)
  print(recommendations)
  test_book_recommendation()

Enter a book name : Where the Heart Is (Oprah's Book Club (Paperback))
["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
